In [12]:
import numpy as np
import pandas as pd
from src.tracking.position_calculation import project_3d_to_2d_left

class KalmanFilter:
    def __init__(self):
        self.x = np.array([[0],
              [0],
              [0],
              [0],
              [0],
              [0],
              [0],
              [0],
              [0]])

        # The initial uncertainty (9x9).
        self.P = np.array([[1000,0,0,0,0,0,0,0,0],
                        [0,1000,0,0,0,0,0,0,0],
                        [0,0,1000,0,0,0,0,0,0],
                        [0,0,0,1000,0,0,0,0,0],
                        [0,0,0,0,1000,0,0,0,0],
                        [0,0,0,0,0,1000,0,0,0],
                        [0,0,0,0,0,0,1000,0,0],
                        [0,0,0,0,0,0,0,1000,0],
                        [0,0,0,0,0,0,0,0,1000]])

        # The external motion (9x1).
        self.u = np.array([[0],
                    [0],
                    [0],
                    [0],
                    [0],
                    [0],
                    [0],
                    [0],
                    [0]])

        # The transition matrix (9x9). 
        self.F = np.array([[1,0,0,1,0,0,0.5,0,0],
                            [0,1,0,0,1,0,0,0.5,0],
                            [0,0,1,0,0,1,0,0,0.5],
                            [0,0,0,1,0,0,1,0,0],
                            [0,0,0,0,1,0,0,1,0],
                            [0,0,0,0,0,1,0,0,1],
                            [0,0,0,0,0,0,1,0,0],
                            [0,0,0,0,0,0,0,1,0],
                            [0,0,0,0,0,0,0,0,1]])

        # The observation matrix (3x9).
        self.H = np.array([[1,0,0,0,0,0,0,0,0],
                    [0,1,0,0,0,0,0,0,0],
                    [0,0,1,0,0,0,0,0,0]])

        # The measurement uncertainty.
        self.R = np.identity(3)

        self.I = np.identity(9)

    def update(self, Z):
        y = Z - np.dot(self.H, self.x)
        S = np.dot(np.dot(self.H, self.P), np.transpose(self.H)) + self.R
        K = np.dot(np.dot(self.P, np.transpose(self.H)), np.linalg.pinv(S))
        self.x = self.x + np.dot(K, y)
        self.P = np.dot(self.I - np.dot(K, self.H), self.P)
        # return self.x, self.P

    def predict(self):
        self.x = np.dot(self.F,self.x) + self.u
        self.P = np.dot(np.dot(self.F, self.P), np.transpose(self.F))
        # return self.x, self.P
    
"""
object_columns = [
        "frame",
        "object_id",
        "object_type",
        "position_x",
        "position_y",
        "position_z",
        "last_seen",
        "bbox_cx",
        "bbox_cy",
        "bbox_w",
        "bbox_h",
    ]
"""

kalman_filter_dict = {}

def kalman_observed_obj(object_id, pos_3d):
    # If object ID has a kalman filter, update it

    if object_id in kalman_filter_dict.keys():
        kalman_filter = kalman_filter_dict[object_id]

    else: 
        kalman_filter = KalmanFilter()
        kalman_filter_dict[object_id] = kalman_filter

    kalman_filter.update(pos_3d)

    ### Predict the next state
    kalman_filter.predict()
    

def kalman_non_observed_obj(object_id):
    # If object ID has a kalman filter, update it

    # assert that a kalman filter exists in the dictionary
    assert object_id in kalman_filter_dict.keys(), "Kalman filter does not exist for object ID: {}".format(object_id)
    kalman_filter = kalman_filter_dict[object_id]

    # ### Predict the next state
    current_pos_3d_x = kalman_filter.x[0][0]
    current_pos_3d_y = kalman_filter.x[0][1]
    current_pos_3d_z = kalman_filter.x[0][2]
    kalman_filter.predict()

    return current_pos_3d_x, current_pos_3d_y, current_pos_3d_z



if __name__ == "__main__":
   
    #read csv file from data folder
    path = "../data/test_final.csv"

    df = pd.read_csv(path)
    df = df.drop(df.columns[0], axis=1)


    # test kalman filter iterating to the first 3 frames
    for frame in range(285):

        print(frame)

        # Iterate through dataframe for i corresponding to frame number
        df_current_frame = df[df["frame"] == frame]
        df_previous_frame = df[df["frame"] == frame-1]


        # OBSERVED OBJECTS

        for index, row in df_current_frame.iterrows():
            x = row["position_x"]
            y = row["position_y"]
            z = row["position_z"]
            kalman_observed_obj(row["object_id"], np.array([x,y,z]))


        # NON-OBSERVED OBJECTS
        
        # find object ids that are not in the current frame but are in the previous frame
        object_ids_previous_frame = df_previous_frame["object_id"].unique()
        object_ids_current_frame = df_current_frame["object_id"].unique()
        object_ids_non_observed = np.setdiff1d(object_ids_previous_frame, object_ids_current_frame)

        rows_to_make = []

        for object_id in object_ids_non_observed:
            est_x, est_y, est_z = kalman_non_observed_obj(object_id)
            object_type = df_previous_frame[df_previous_frame["object_id"] == object_id]["object_type"].values[0]
            last_seen = df_previous_frame[df_previous_frame["object_id"] == object_id]["last_seen"].values[0]
            bbox_w = df_previous_frame[df_previous_frame["object_id"] == object_id]["bbox_w"].values[0]
            bbox_h = df_previous_frame[df_previous_frame["object_id"] == object_id]["bbox_h"].values[0]
            bbox_cx, bbox_cy = project_3d_to_2d_left([est_x, est_y, est_z])

            # Make new row for non-observed object in dataframe
            row = {
                "object_id": object_id,
                "object_type": object_type,
                "position_x": est_x,
                "position_y": est_y,
                "position_z": est_z,
                "last_seen": last_seen,
                "bbox_cx": bbox_cx,
                "bbox_cy": bbox_cy,
                "bbox_w": bbox_w,
                "bbox_h": bbox_h
                }
            rows_to_make.append(row)
            print(row)

       

0
1
{'object_id': '89102a72-3703-4e49-8a7e-a7a4c11eb588', 'object_type': 'Car', 'position_x': -46.90482242008605, 'position_y': 10.223807293304167, 'position_z': 126.72452915194022, 'last_seen': 0, 'bbox_cx': 1141.8973388671875, 'bbox_cy': 167.47008514404297, 'bbox_w': 136.986572265625, 'bbox_h': 52.50883483886719}
2
{'object_id': '00787492-9fd5-475a-8599-55d3f9baa464', 'object_type': 'Pedestrian', 'position_x': 6.540061697204035, 'position_y': 0.3360720093174248, 'position_z': 15.223346904435354, 'last_seen': 1, 'bbox_cx': 159.1294191467656, 'bbox_cy': 230.41821377224468, 'bbox_w': 56.57997894287109, 'bbox_h': 129.3420867919922}
3
4
5
6
{'object_id': '38f62538-20fe-4fae-814b-d2398f3d3156', 'object_type': 'Car', 'position_x': -2.1027089032548223, 'position_y': 0.4208632243461251, 'position_z': 12.545951176653118, 'last_seen': 5, 'bbox_cx': 893.014404296875, 'bbox_cy': 218.09779357910156, 'bbox_w': 49.90277099609375, 'bbox_h': 120.48876953125}
7
8
{'object_id': '89102a72-3703-4e49-8a7e-